In [1]:
from pipeline import *

(CVXPY) Jun 24 05:00:07 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.6.2534). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jun 24 05:00:07 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.6.2534). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from catboost import CatBoostClassifier

np.random.seed(101)

In [3]:
df = pd.read_parquet('/Users/mac/PycharmProjects/MLOps /data/phase-1/prob-2/raw_train.parquet')

In [4]:
df = df.sample(frac=1)

In [5]:
df.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,label
5995,V2,11.645719,V4,V4,2617.519333,V0,V3,4.0,V0,V0,...,V0,32.037193,none,V1,2.0,V2,2.0,yes,yes,0
17823,V1,24.351003,V4,V1,3665.757647,V0,V2,4.0,V2,V0,...,V2,41.756374,none,V1,1.0,V1,1.0,none,yes,0
33606,V0,17.826815,V4,V3,3989.534404,V0,V1,2.0,V2,V0,...,V2,22.011693,bank,V1,3.0,V3,2.0,yes,yes,0
48301,V3,24.964412,V2,V0,856.244244,V3,V4,4.0,V2,V0,...,V0,56.731349,none,V0,1.0,V2,2.0,none,yes,0
47869,V1,11.750023,V2,V0,1515.905359,V0,V3,2.0,V1,V0,...,V2,32.896634,none,V1,1.0,V2,1.0,yes,yes,1


In [6]:
train = df.drop(columns=['label'])
label = df['label']

In [7]:
X_train, X_val, y_train, y_val = train_test_split(train, label, test_size=0.2, random_state=101)

In [8]:
cats = ["feature1",
        "feature3",
        "feature4",
        "feature6",
        "feature7",
        "feature8",
        "feature9",
        "feature10",
        "feature11",
        "feature12",
        "feature14",
        "feature15",
        "feature16",
        "feature17",
        "feature19",
        "feature20"]

In [9]:
from sklearn.pipeline import FeatureUnion, Pipeline

In [10]:
columns = ['feature{}'.format(i) for i in range(1, 21)]

In [11]:
s = Pipeline(
    steps=(
        ('feature_select', FeatureSelector(columns)),
        ('woe', WOE(cats=cats, nums=[]))
    )
)

In [12]:
s.fit(X_train, y_train)

Pipeline(steps=[('feature_select',
                 FeatureSelector(_feature_names=['feature1', 'feature2',
                                                 'feature3', 'feature4',
                                                 'feature5', 'feature6',
                                                 'feature7', 'feature8',
                                                 'feature9', 'feature10',
                                                 'feature11', 'feature12',
                                                 'feature13', 'feature14',
                                                 'feature15', 'feature16',
                                                 'feature17', 'feature18',
                                                 'feature19', 'feature20'])),
                ('woe',
                 WOE(cats=['feature1', 'feature3', 'feature4', 'feature6',
                           'feature7', 'feature8', 'feature9', 'feature10',
                           'feature11', 'feature12', 'feature14', 'feature15',
                           'feature16', 'feature17', 'feature19', 'feature20'],
                     nums=[]))])

In [13]:
import pickle

with open('/Users/mac/PycharmProjects/MLOps /src/phase1/models/pipeline_2.pkl', 'wb') as f:
    pickle.dump(s, f)

In [14]:
X_train = s.transform(X_train)
X_val = s.transform(X_val)

In [15]:
model = CatBoostClassifier(
    iterations=2000,
    random_seed=101,
    learning_rate=0.02,
    eval_metric='AUC'
)

In [16]:
model.fit(X_train, y_train, eval_set=(X_val, y_val))

0:	test: 0.7851789	best: 0.7851789 (0)	total: 71.9ms	remaining: 2m 23s
1:	test: 0.7959321	best: 0.7959321 (1)	total: 80ms	remaining: 1m 19s
2:	test: 0.8003956	best: 0.8003956 (2)	total: 87.3ms	remaining: 58.1s
3:	test: 0.8044563	best: 0.8044563 (3)	total: 92.9ms	remaining: 46.4s
4:	test: 0.8061012	best: 0.8061012 (4)	total: 99.5ms	remaining: 39.7s
5:	test: 0.8052747	best: 0.8061012 (4)	total: 105ms	remaining: 34.9s
6:	test: 0.8051344	best: 0.8061012 (4)	total: 110ms	remaining: 31.4s
7:	test: 0.8069179	best: 0.8069179 (7)	total: 116ms	remaining: 29s
8:	test: 0.8076717	best: 0.8076717 (8)	total: 122ms	remaining: 27s
9:	test: 0.8076318	best: 0.8076717 (8)	total: 128ms	remaining: 25.4s
10:	test: 0.8087272	best: 0.8087272 (10)	total: 133ms	remaining: 24.1s
11:	test: 0.8086511	best: 0.8087272 (10)	total: 139ms	remaining: 23.1s
12:	test: 0.8091594	best: 0.8091594 (12)	total: 145ms	remaining: 22.1s
13:	test: 0.8099563	best: 0.8099563 (13)	total: 150ms	remaining: 21.2s
14:	test: 0.8097986	best:

141:	test: 0.8385622	best: 0.8385682 (140)	total: 865ms	remaining: 11.3s
142:	test: 0.8386423	best: 0.8386423 (142)	total: 871ms	remaining: 11.3s
143:	test: 0.8388591	best: 0.8388591 (143)	total: 878ms	remaining: 11.3s
144:	test: 0.8389022	best: 0.8389022 (144)	total: 885ms	remaining: 11.3s
145:	test: 0.8390715	best: 0.8390715 (145)	total: 891ms	remaining: 11.3s
146:	test: 0.8393868	best: 0.8393868 (146)	total: 896ms	remaining: 11.3s
147:	test: 0.8397218	best: 0.8397218 (147)	total: 901ms	remaining: 11.3s
148:	test: 0.8399201	best: 0.8399201 (148)	total: 906ms	remaining: 11.3s
149:	test: 0.8399858	best: 0.8399858 (149)	total: 912ms	remaining: 11.3s
150:	test: 0.8400637	best: 0.8400637 (150)	total: 918ms	remaining: 11.2s
151:	test: 0.8401557	best: 0.8401557 (151)	total: 923ms	remaining: 11.2s
152:	test: 0.8402357	best: 0.8402357 (152)	total: 929ms	remaining: 11.2s
153:	test: 0.8403617	best: 0.8403617 (153)	total: 934ms	remaining: 11.2s
154:	test: 0.8403968	best: 0.8403968 (154)	total: 9

282:	test: 0.8524071	best: 0.8524071 (282)	total: 1.66s	remaining: 10.1s
283:	test: 0.8525154	best: 0.8525154 (283)	total: 1.66s	remaining: 10.1s
284:	test: 0.8525893	best: 0.8525893 (284)	total: 1.67s	remaining: 10.1s
285:	test: 0.8526398	best: 0.8526398 (285)	total: 1.68s	remaining: 10s
286:	test: 0.8527914	best: 0.8527914 (286)	total: 1.68s	remaining: 10s
287:	test: 0.8527777	best: 0.8527914 (286)	total: 1.69s	remaining: 10s
288:	test: 0.8528693	best: 0.8528693 (288)	total: 1.69s	remaining: 10s
289:	test: 0.8528868	best: 0.8528868 (289)	total: 1.7s	remaining: 10s
290:	test: 0.8528801	best: 0.8528868 (289)	total: 1.7s	remaining: 9.99s
291:	test: 0.8528930	best: 0.8528930 (291)	total: 1.71s	remaining: 9.98s
292:	test: 0.8529312	best: 0.8529312 (292)	total: 1.71s	remaining: 9.97s
293:	test: 0.8530264	best: 0.8530264 (293)	total: 1.72s	remaining: 9.96s
294:	test: 0.8530711	best: 0.8530711 (294)	total: 1.72s	remaining: 9.95s
295:	test: 0.8530734	best: 0.8530734 (295)	total: 1.73s	remaini

433:	test: 0.8600457	best: 0.8600457 (433)	total: 2.46s	remaining: 8.88s
434:	test: 0.8600819	best: 0.8600819 (434)	total: 2.47s	remaining: 8.87s
435:	test: 0.8600747	best: 0.8600819 (434)	total: 2.47s	remaining: 8.87s
436:	test: 0.8601238	best: 0.8601238 (436)	total: 2.48s	remaining: 8.86s
437:	test: 0.8601953	best: 0.8601953 (437)	total: 2.48s	remaining: 8.85s
438:	test: 0.8602684	best: 0.8602684 (438)	total: 2.49s	remaining: 8.84s
439:	test: 0.8604234	best: 0.8604234 (439)	total: 2.49s	remaining: 8.83s
440:	test: 0.8604997	best: 0.8604997 (440)	total: 2.5s	remaining: 8.83s
441:	test: 0.8605088	best: 0.8605088 (441)	total: 2.5s	remaining: 8.82s
442:	test: 0.8605092	best: 0.8605092 (442)	total: 2.51s	remaining: 8.81s
443:	test: 0.8606290	best: 0.8606290 (443)	total: 2.51s	remaining: 8.81s
444:	test: 0.8606381	best: 0.8606381 (444)	total: 2.52s	remaining: 8.8s
445:	test: 0.8606730	best: 0.8606730 (445)	total: 2.52s	remaining: 8.79s
446:	test: 0.8606670	best: 0.8606730 (445)	total: 2.53

546:	test: 0.8641034	best: 0.8641034 (546)	total: 3.06s	remaining: 8.13s
547:	test: 0.8640949	best: 0.8641034 (546)	total: 3.07s	remaining: 8.13s
548:	test: 0.8641591	best: 0.8641591 (548)	total: 3.07s	remaining: 8.12s
549:	test: 0.8641811	best: 0.8641811 (549)	total: 3.08s	remaining: 8.12s
550:	test: 0.8641936	best: 0.8641936 (550)	total: 3.08s	remaining: 8.11s
551:	test: 0.8642052	best: 0.8642052 (551)	total: 3.09s	remaining: 8.11s
552:	test: 0.8643071	best: 0.8643071 (552)	total: 3.1s	remaining: 8.1s
553:	test: 0.8643226	best: 0.8643226 (553)	total: 3.1s	remaining: 8.09s
554:	test: 0.8643810	best: 0.8643810 (554)	total: 3.1s	remaining: 8.09s
555:	test: 0.8644970	best: 0.8644970 (555)	total: 3.11s	remaining: 8.08s
556:	test: 0.8645481	best: 0.8645481 (556)	total: 3.12s	remaining: 8.07s
557:	test: 0.8645772	best: 0.8645772 (557)	total: 3.12s	remaining: 8.07s
558:	test: 0.8645765	best: 0.8645772 (557)	total: 3.13s	remaining: 8.06s
559:	test: 0.8646422	best: 0.8646422 (559)	total: 3.13s

...

1438:	test: 0.8749005	best: 0.8749801 (1404)	total: 7.86s	remaining: 3.06s
1439:	test: 0.8749168	best: 0.8749801 (1404)	total: 7.87s	remaining: 3.06s
1440:	test: 0.8749115	best: 0.8749801 (1404)	total: 7.88s	remaining: 3.06s
1441:	test: 0.8749157	best: 0.8749801 (1404)	total: 7.88s	remaining: 3.05s
1442:	test: 0.8749110	best: 0.8749801 (1404)	total: 7.89s	remaining: 3.04s
1443:	test: 0.8749304	best: 0.8749801 (1404)	total: 7.89s	remaining: 3.04s
1444:	test: 0.8749418	best: 0.8749801 (1404)	total: 7.9s	remaining: 3.03s
1445:	test: 0.8749488	best: 0.8749801 (1404)	total: 7.9s	remaining: 3.03s
1446:	test: 0.8749478	best: 0.8749801 (1404)	total: 7.91s	remaining: 3.02s
1447:	test: 0.8749508	best: 0.8749801 (1404)	total: 7.91s	remaining: 3.02s
1448:	test: 0.8749478	best: 0.8749801 (1404)	total: 7.92s	remaining: 3.01s
1449:	test: 0.8749487	best: 0.8749801 (1404)	total: 7.92s	remaining: 3s
1450:	test: 0.8749623	best: 0.8749801 (1404)	total: 7.93s	remaining: 3s
1451:	test: 0.8749654	best: 0.874

1550:	test: 0.8752165	best: 0.8752270 (1548)	total: 8.47s	remaining: 2.45s
1551:	test: 0.8752112	best: 0.8752270 (1548)	total: 8.47s	remaining: 2.45s
1552:	test: 0.8752069	best: 0.8752270 (1548)	total: 8.48s	remaining: 2.44s
1553:	test: 0.8752105	best: 0.8752270 (1548)	total: 8.48s	remaining: 2.44s
1554:	test: 0.8752401	best: 0.8752401 (1554)	total: 8.49s	remaining: 2.43s
1555:	test: 0.8752335	best: 0.8752401 (1554)	total: 8.5s	remaining: 2.42s
1556:	test: 0.8752445	best: 0.8752445 (1556)	total: 8.5s	remaining: 2.42s
1557:	test: 0.8752452	best: 0.8752452 (1557)	total: 8.51s	remaining: 2.41s
1558:	test: 0.8752410	best: 0.8752452 (1557)	total: 8.51s	remaining: 2.41s
1559:	test: 0.8752373	best: 0.8752452 (1557)	total: 8.52s	remaining: 2.4s
1560:	test: 0.8752311	best: 0.8752452 (1557)	total: 8.52s	remaining: 2.4s
1561:	test: 0.8752278	best: 0.8752452 (1557)	total: 8.53s	remaining: 2.39s
1562:	test: 0.8752212	best: 0.8752452 (1557)	total: 8.54s	remaining: 2.39s
1563:	test: 0.8752207	best: 0

1691:	test: 0.8753361	best: 0.8753904 (1677)	total: 9.27s	remaining: 1.69s
1692:	test: 0.8753332	best: 0.8753904 (1677)	total: 9.28s	remaining: 1.68s
1693:	test: 0.8753269	best: 0.8753904 (1677)	total: 9.29s	remaining: 1.68s
1694:	test: 0.8753178	best: 0.8753904 (1677)	total: 9.29s	remaining: 1.67s
1695:	test: 0.8753182	best: 0.8753904 (1677)	total: 9.3s	remaining: 1.67s
1696:	test: 0.8753244	best: 0.8753904 (1677)	total: 9.3s	remaining: 1.66s
1697:	test: 0.8753273	best: 0.8753904 (1677)	total: 9.31s	remaining: 1.66s
1698:	test: 0.8753304	best: 0.8753904 (1677)	total: 9.32s	remaining: 1.65s
1699:	test: 0.8753197	best: 0.8753904 (1677)	total: 9.32s	remaining: 1.65s
1700:	test: 0.8753212	best: 0.8753904 (1677)	total: 9.33s	remaining: 1.64s
1701:	test: 0.8753198	best: 0.8753904 (1677)	total: 9.33s	remaining: 1.63s
1702:	test: 0.8753240	best: 0.8753904 (1677)	total: 9.34s	remaining: 1.63s
1703:	test: 0.8753272	best: 0.8753904 (1677)	total: 9.34s	remaining: 1.62s
1704:	test: 0.8753262	best:

1835:	test: 0.8754229	best: 0.8754413 (1832)	total: 10.1s	remaining: 900ms
1836:	test: 0.8754287	best: 0.8754413 (1832)	total: 10.1s	remaining: 895ms
1837:	test: 0.8754241	best: 0.8754413 (1832)	total: 10.1s	remaining: 889ms
1838:	test: 0.8754226	best: 0.8754413 (1832)	total: 10.1s	remaining: 884ms
1839:	test: 0.8754279	best: 0.8754413 (1832)	total: 10.1s	remaining: 879ms
1840:	test: 0.8754196	best: 0.8754413 (1832)	total: 10.1s	remaining: 873ms
1841:	test: 0.8754289	best: 0.8754413 (1832)	total: 10.1s	remaining: 868ms
1842:	test: 0.8754255	best: 0.8754413 (1832)	total: 10.1s	remaining: 862ms
1843:	test: 0.8754319	best: 0.8754413 (1832)	total: 10.1s	remaining: 857ms
1844:	test: 0.8754426	best: 0.8754426 (1844)	total: 10.1s	remaining: 851ms
1845:	test: 0.8754393	best: 0.8754426 (1844)	total: 10.1s	remaining: 846ms
1846:	test: 0.8754614	best: 0.8754614 (1846)	total: 10.1s	remaining: 840ms
1847:	test: 0.8754707	best: 0.8754707 (1847)	total: 10.1s	remaining: 835ms
1848:	test: 0.8754725	bes

1980:	test: 0.8755862	best: 0.8756375 (1945)	total: 10.9s	remaining: 104ms
1981:	test: 0.8755819	best: 0.8756375 (1945)	total: 10.9s	remaining: 98.8ms
1982:	test: 0.8755851	best: 0.8756375 (1945)	total: 10.9s	remaining: 93.4ms
1983:	test: 0.8755840	best: 0.8756375 (1945)	total: 10.9s	remaining: 87.9ms
1984:	test: 0.8755828	best: 0.8756375 (1945)	total: 10.9s	remaining: 82.4ms
1985:	test: 0.8755812	best: 0.8756375 (1945)	total: 10.9s	remaining: 76.9ms
1986:	test: 0.8755877	best: 0.8756375 (1945)	total: 10.9s	remaining: 71.4ms
1987:	test: 0.8755825	best: 0.8756375 (1945)	total: 10.9s	remaining: 65.9ms
1988:	test: 0.8755825	best: 0.8756375 (1945)	total: 10.9s	remaining: 60.4ms
1989:	test: 0.8755819	best: 0.8756375 (1945)	total: 10.9s	remaining: 54.9ms
1990:	test: 0.8755864	best: 0.8756375 (1945)	total: 10.9s	remaining: 49.4ms
1991:	test: 0.8755868	best: 0.8756375 (1945)	total: 10.9s	remaining: 43.9ms
1992:	test: 0.8755870	best: 0.8756375 (1945)	total: 10.9s	remaining: 38.4ms
1993:	test: 0

In [17]:
import pickle

with open('/Users/mac/PycharmProjects/MLOps /src/phase1/models/problem2.pkl', 'wb') as f:
    pickle.dump(model, f)

In [8]:
train.values[:5]

array([['V2', 11.645719, 'V4', 'V4', 2617.519333, 'V0', 'V3', 4.0, 'V0',
        'V0', 3.0, 'V0', 32.037193, 'none', 'V1', 2.0, 'V2', 2.0, 'yes',
        'yes'],
       ['V1', 24.351003, 'V4', 'V1', 3665.757647, 'V0', 'V2', 4.0, 'V2',
        'V0', 4.0, 'V2', 41.756374, 'none', 'V1', 1.0, 'V1', 1.0, 'none',
        'yes'],
       ['V0', 17.826815, 'V4', 'V3', 3989.534404, 'V0', 'V1', 2.0, 'V2',
        'V0', 4.0, 'V2', 22.011693, 'bank', 'V1', 3.0, 'V3', 2.0, 'yes',
        'yes'],
       ['V3', 24.964412, 'V2', 'V0', 856.244244, 'V3', 'V4', 4.0, 'V2',
        'V0', 2.0, 'V0', 56.731349, 'none', 'V0', 1.0, 'V2', 2.0, 'none',
        'yes'],
       ['V1', 11.750023, 'V2', 'V0', 1515.905359, 'V0', 'V3', 2.0, 'V1',
        'V0', 3.0, 'V2', 32.896634, 'none', 'V1', 1.0, 'V2', 1.0, 'yes',
        'yes']], dtype=object)